In [66]:
import pandas as pd
import os
from gscripts import qtools
import pandas as pd
from gscripts.encode import encode_helpers
import functools
from tqdm import tqdm, tqdm_notebook
tqdm.pandas("Progress: ")


In [8]:
merged_data = encode_helpers.get_merged_data()

#For Peak Analysis we only want to analyze datasets that have been submitted
merged_data = merged_data[merged_data.submitted]
merged_data = merged_data[['CLIP', 'INPUT', 'input_norm']]

In [138]:
def get_gps_peak(fn):
    gps_dir = "/projects/ps-yeolab3/encode/analysis/gps_peaks"
    fn = os.path.basename(fn)
    basename = fn.split(".")[0]
    
    fn = os.path.join(gps_dir, "{}/{}_outputs/{}_2_GEM_events.bed".format(basename, basename, basename))
    return fn

def get_gps_peak_loose(fn):
    gps_dir = "/projects/ps-yeolab3/encode/analysis/gps_peaks_loose"
    fn = os.path.basename(fn)
    basename = fn.split(".")[0]
    
    fn = os.path.join(gps_dir, "{}/{}.GEM_events.bed".format(basename, basename, basename))
    return fn

In [167]:
results = []
for name, row in merged_data.iterrows():
    if not all([os.path.exists(row.CLIP), os.path.exists(row.INPUT)]):
        print "row doesn't exist", row.CLIP
        
    if not os.path.exists(get_gps_peak(row.CLIP)):
        result_line = "/home/gpratt/ad-hoc/gem/run_gem.sh {} {}".format(row.CLIP, row.INPUT)
        results.append(result_line)
        
    if not os.path.exists(get_gps_peak_loose(row.CLIP)):
        result_line = "/home/gpratt/ad-hoc/gem/run_gem_loose.sh {} {}".format(row.CLIP, row.INPUT)
        results.append(result_line)

In [168]:
job_name = "gps_peaks"
script_dir = "/home/gpratt/projects/encode/scripts/"
job = qtools.Submitter(commands=results, 
                 job_name="{}".format(job_name), 
                 sh_filename= os.path.join(script_dir, "{}.sh".format(job_name)),
                array=True,
                walltime="8:00:00",
                out_filename= os.path.join(script_dir, "{}.out".format(job_name)),
                err_filename=os.path.join(script_dir, "{}.err".format(job_name)),
                queue="hotel")
job.job()

print os.path.join(script_dir, "{}.sh".format(job_name))

/home/gpratt/projects/encode/scripts/gps_peaks.sh


running 0 tasks as an array-job.


# IDR on output

In [169]:
def get_gps_peak(fn, gps_dir="/projects/ps-yeolab3/encode/analysis/gps_peaks"):
    fn = os.path.basename(fn)
    basename = fn.split(".")[0]
    
    fn = os.path.join(gps_dir, "{}/{}_outputs/{}_2_GEM_events.bed".format(basename, basename, basename))
    return fn

def get_gps_events(fn, gps_dir="/projects/ps-yeolab3/encode/analysis/gps_peaks"):
    fn = os.path.basename(fn)
    basename = fn.split(".")[0]
    
    fn = os.path.join(gps_dir, "{}/{}_outputs/{}_2_GEM_events.txt".format(basename, basename, basename))
    return fn

def get_gps_peak_v3(fn, gps_dir="/projects/ps-yeolab3/encode/analysis/gps_peaks_loose/"):
    fn = os.path.basename(fn)
    basename = fn.split(".")[0]
    
    fn = os.path.join(gps_dir, "{}/{}.GEM_events.bed".format(basename, basename, basename))
    return fn

def get_gps_events_v3(fn, gps_dir="/projects/ps-yeolab3/encode/analysis/gps_peaks_loose/"):
    fn = os.path.basename(fn)
    basename = fn.split(".")[0]
    
    fn = os.path.join(gps_dir, "{}/{}.GEM_events.txt".format(basename, basename, basename))
    return fn

In [172]:
merged_data['gps'] = merged_data.CLIP.apply(get_gps_peak)
merged_data['gps_txt'] = merged_data.CLIP.apply(get_gps_events)


merged_data['gps_loose'] = merged_data.CLIP.apply(get_gps_peak_v3)
merged_data['gps_txt_loose'] = merged_data.CLIP.apply(get_gps_events_v3)

In [173]:
def make_narrowPeak_from_gps(row, add_loose=False):
    gps = "gps"
    gps_txt = "gps_txt"
    
    if add_loose:
        gps += "_loose"
        gps_txt += "_loose"
        
    out_file = os.path.splitext(row[gps])[0] + '.narrowPeak.bed'
    if os.path.exists(out_file):
        return out_file
    
    bed = pd.read_table(row[gps], skiprows=1, 
                        names=['chrom', 'start', 'stop', 'Position', 'IP', 'null', 'strand'])
    bed['strand_position'] = bed.Position + ":" + bed.strand

    events = pd.read_table(row[gps_txt])

    full_data = pd.merge(bed, events, left_on="strand_position", right_on="Position")

    if not (len(full_data) == len(bed) == len(events)):
        print "error"

    narrowpeak_df = full_data[['chrom', 'start', 'stop', 'Position_x', 'IP_x', 'strand', 'Fold', 'P_-lg10', 'Q_-lg10', 'Position_y']]
    narrowpeak_df.to_csv(out_file, sep="\t", header=False, index=False)
    return out_file

In [174]:
def make_narrowPeak_from_gps_v3(row, add_loose=False):
    gps = "gps"
    gps_txt = "gps_txt"
    
    if add_loose:
        gps += "_loose"
        gps_txt += "_loose"
        
    out_file = os.path.splitext(row[gps])[0] + '.narrowPeak.bed'
    if os.path.exists(out_file):
        return out_file
    
    bed = pd.read_table(row[gps], skiprows=1, 
                        names=['chrom', 'start', 'stop', 'Position', 'IP', 'null', 'strand'])
    bed['strand_position'] = bed.Position + ":" + bed.strand

    events = pd.read_table(row[gps_txt])

    full_data = pd.merge(bed, events, left_on="strand_position", right_on="Position")

    if not (len(full_data) == len(bed) == len(events)):
        print "error"

    narrowpeak_df = full_data[['chrom', 'start', 'stop', 'Position_x', 'IP_x', 'strand', 'Fold', 'P_-lg10', 'Q_-lg10', 'Position_y']]
    narrowpeak_df.to_csv(out_file, sep="\t", header=False, index=False)
    return out_file

In [175]:
merged_data['gps_narrowPeak'] = merged_data.progress_apply(make_narrowPeak_from_gps, axis=1)

make_narrowPeak_from_gps_loose = functools.partial(make_narrowPeak_from_gps, add_loose=True)
merged_data['gps_narrowPeak_loose'] = merged_data.progress_apply(make_narrowPeak_from_gps_loose, axis=1)



  0%|          | 0/9 [00:00<?, ?it/s]

 11%|█         | 1/9 [24:25:57<195:27:38, 87957.30s/it]

 33%|███▎      | 3/9 [00:01<00:03,  1.52it/s]

 44%|████▍     | 4/9 [00:02<00:03,  1.64it/s]

 56%|█████▌    | 5/9 [00:03<00:02,  1.73it/s]

 67%|██████▋   | 6/9 [00:03<00:01,  1.87it/s]

 78%|███████▊  | 7/9 [00:04<00:01,  1.80it/s]

 89%|████████▉ | 8/9 [00:04<00:00,  1.83it/s]

100%|██████████| 9/9 [00:05<00:00,  1.94it/s]

10it [00:05,  2.50it/s]                      

12it [00:05,  2.56it/s]

13it [00:06,  2.40it/s]

14it [00:06,  2.68it/s]

15it [00:07,  2.54it/s]

16it [00:07,  2.59it/s]

17it [00:07,  3.13it/s]

19it [00:07,  3.79it/s]

20it [00:08,  2.60it/s]

21it [00:09,  2.36it/s]

22it [00:10,  1.69it/s]

23it [00:10,  1.44it/s]

24it [00:11,  1.91it/s]

25it [00:11,  2.36it/s]

26it [00:11,  2.07it/s]

27it [00:12,  1.93it/s]

28it [00:13,  1.95it/s]

29it [00:13,  1.85it/s]

31it [00:13,  2.30it/s]

32it [00:14,  2.28it/s]

33it [00:14,  2.26it/s]

34it [00:15,  2.75it/s]

3

In [182]:
out_dir = "/projects/ps-yeolab3/encode/analysis/gps_peaks/"
def submit_idr(df):
    out_file = os.path.join(out_dir, "_".join(df.index[0][:-1]) + ".idr.peaks.bed")
    rep1 = df.iloc[0].gps_narrowPeak
    rep2 = df.iloc[1].gps_narrowPeak
    out_file = os.path.join(out_file)
    out_cmd = idr(rep1, rep2, out_file)
    return out_cmd

def submit_idr_loose(df):
    out_file = os.path.join(out_dir, "_".join(df.index[0][:-1]) + ".idr.peaks_loose.bed")
    rep1 = df.iloc[0].gps_narrowPeak_loose
    rep2 = df.iloc[1].gps_narrowPeak_loose
    out_file = os.path.join(out_file)
    out_cmd = idr(rep1, rep2, out_file)
    return out_cmd

def idr(peak_1, peak_2, out_file):
    return "source activate py3 && idr  '--samples'  '{}'  '{}'  '--input-file-type' 'bed'  '--rank' '7'  '--peak-merge-method' 'max'  '--plot'  '-o' '{}'".format(peak_1, peak_2, out_file)


In [183]:
grp = merged_data.groupby(level=["uID", 'Cell line', 'RBP'])
samples_to_run = grp.apply(submit_idr_loose).values

In [184]:

job_name = "gps_idr"
script_dir = "/home/gpratt/projects/encode/scripts/"
job = qtools.Submitter(commands=samples_to_run, 
                 job_name="{}".format(job_name), 
                 sh_filename= os.path.join(script_dir, "{}.sh".format(job_name)),
                array=True,
                walltime="8:00:00",
                out_filename= os.path.join(script_dir, "{}.out".format(job_name)),
                err_filename=os.path.join(script_dir, "{}.err".format(job_name)),
                queue="hotel")
job.job()

print os.path.join(script_dir, "{}.sh".format(job_name))

/home/gpratt/projects/encode/scripts/gps_idr.sh


running 181 tasks as an array-job.
